In [ ]:
import os
import json
import statistics
from collections import Counter

import pandas as pd
import seaborn as sns
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

_ = nltk.download("stopwords", quiet=True)
_ = nltk.download("punkt", quiet=True)

In [ ]:
# Create the data/02_postprocessing direction if it does not exist
if not os.path.exists("data/02_postprocessing"):
    os.makedirs("data/02_postprocessing")
# Create the data/02_postprocessing/sample directory if it does not exist
if not os.path.exists("data/02_postprocessing/sample"):
    os.makedirs("data/02_postprocessing/sample")

In [ ]:
def silent_remove(filename):
    try:
        os.remove(filename)
    except OSError:
        pass

This notebook is designed so that it can be run on the sample data or the full data just by switching one variable.

In [ ]:
# Set this to True to run the script on the sample data
# Set this to False to run the script on the full data (takes much longer)
SAMPLE = False

## Ratings

This section performs rating normalization (technically standardization) to all ratings

In [ ]:
# [!] 4 minutes to load
with open(f"data/01_cleaned/{'sample/' if SAMPLE else ''}ratings.json") as f:
    ratings_original = [json.loads(line) for line in f]
with open(f"data/01_cleaned/{'sample/' if SAMPLE else ''}users.json") as f:
    users_original = [json.loads(line) for line in f]

In [ ]:
user_ratings_dict = {}
for rating in ratings_original:
    user_id = rating["user_id"]
    if user_id not in user_ratings_dict.keys():
        user_ratings_dict[user_id] = {"rating_objs": [rating], "ratings": [(rating["rating_id"], rating["stars"])]}
    else:
        user_ratings_dict[user_id]["rating_objs"].append(rating)
        user_ratings_dict[user_id]["ratings"].append((rating["rating_id"], rating["stars"]))

In [ ]:
# Used to determine the cut off for how many ratings a user should have before we
# apply z-score normalization

# This outputs the ratio between the number of ratings that would be standardized
# and all ratings in the dataset

cutoff = 5
num_ratings = [len(user["ratings"]) for user in user_ratings_dict.values()]
rating_count = Counter(num_ratings)
rating_counter_agg = [(tup[0], tup[1], tup[0] * tup[1]) for tup in rating_count.items()]
ratio = sum([tup[2] for tup in rating_counter_agg if tup[0] >= cutoff]) / sum([tup[2] for tup in rating_counter_agg])
ratio

In [ ]:
users_not_standardized = []
users_standardized = []
for user_id, user_ratings in user_ratings_dict.items():
    if (
        len(user_ratings["ratings"]) >= cutoff
        and len(set(list(zip(*user_ratings["ratings"]))[1])) >= 2
    ):
        users_standardized.append(user_id)
        user_ratings["avg_rating"] = statistics.mean(
            list(zip(*user_ratings["ratings"]))[1]
        )
        user_ratings["std_dev"] = statistics.stdev(
            list(zip(*user_ratings["ratings"]))[1]
        )
        user_ratings["standardized_ratings"] = [
            (
                rating[0],
                (rating[1] - user_ratings["avg_rating"]) / user_ratings["std_dev"],
            )
            for rating in user_ratings["ratings"]
        ]
    else:
        users_not_standardized.append(user_id)

In [ ]:
# Gather all ratings standardized by user
user_standardized_ratings = [
    rating
    for user_ratings in [
        user_rating
        for user_rating in user_ratings_dict.values()
        if "standardized_ratings" in user_rating.keys()
    ]
    for rating in user_ratings["standardized_ratings"]
]

In [ ]:
# standardize all other ratings with respect to each other
not_standardized_ratings = [
    rating
    for user_ratings in [
        user_ratings_dict[user_id] for user_id in users_not_standardized
    ]
    for rating in user_ratings["ratings"]
]
mean = statistics.mean(list(zip(*not_standardized_ratings))[1])
std_dev = statistics.stdev(list(zip(*not_standardized_ratings))[1])
pool_standardized_ratings = [
    (rating[0], (rating[1] - mean) / std_dev) for rating in not_standardized_ratings
]

In [ ]:
ratings = {rating["rating_id"]: rating for rating in ratings_original}
for rating in user_standardized_ratings + pool_standardized_ratings:
    ratings[rating[0]]["standardized_rating"] = rating[1]

In [ ]:
# Add a min-max normalized standardized rating for each rating object
# The attribute will be called "norm_std_rating"

rating_df = pd.DataFrame(ratings_original).drop(
    columns=["business_id", "rating_id"]
)
max_std_rating = rating_df["standardized_rating"].max()
min_std_rating = rating_df["standardized_rating"].min()

rating_df["norm_std_rating"] = (
    (rating_df["standardized_rating"] - min_std_rating) /
    (max_std_rating - min_std_rating)
)
rating_df["norm_std_rating"] = (rating_df["norm_std_rating"] * 4) + 1
rating_df.head()

In [ ]:
# Make standardized_rating vs norm_std_rating a yes-no attribute
# Make it wide-form data
rating_df_wide_list = []
for index, row in rating_df.iterrows():
    rating_df_wide_list.append({
        "user_id": row["user_id"],
        "standardized": "Yes",
        "rating": row["norm_std_rating"],
    })
    rating_df_wide_list.append({
        "user_id": row["user_id"],
        "standardized": "No",
        "rating": row["stars"],
    })
rating_df_wide = pd.DataFrame(rating_df_wide_list)
rating_df_wide.head()

In [ ]:
# Find the user with the max average rating and the user with the min average rating

graphing_cutoff = 20
graph_pool = [
    user
    for user in users_standardized
    if len(user_ratings_dict[user]["ratings"]) >= graphing_cutoff
]
print(f"Narrowed down to {len(graph_pool) * 100 / len(users_standardized)}%")

max_avg_user = max(users_standardized, key=lambda x: user_ratings_dict[x]["avg_rating"])
min_avg_user = min(users_standardized, key=lambda x: user_ratings_dict[x]["avg_rating"])
max_std_dev_user = max(
    users_standardized, key=lambda x: user_ratings_dict[x]["std_dev"]
)
min_std_dev_user = min(
    list(set(users_standardized) - set([max_avg_user])), key=lambda x: user_ratings_dict[x]["std_dev"]
)
print(f"Found 4 users: {max_avg_user}, {min_avg_user}, {max_std_dev_user}, {min_std_dev_user}")

In [ ]:
graph_df = rating_df_wide.copy()
graph_df["user_id"] = graph_df["user_id"].map({
    max_avg_user: "max_avg",
    min_avg_user: "min_avg",
    max_std_dev_user: "max_std_dev",
    min_std_dev_user: "min_std_dev",
}).fillna("other")
graph_df.head()

In [ ]:
_ = sns.displot(
    data=graph_df.loc[graph_df["user_id"] != "other"],
    kind="hist",
    x="rating",
    hue="standardized",
    col="user_id",
    col_order=["max_avg", "min_avg", "max_std_dev", "min_std_dev"],
    col_wrap=2,
    stat="probability",
    common_norm=False,
    bins=10,
)

In [ ]:
_ = sns.displot(
    data=graph_df,
    kind="hist",
    x="rating",
    hue="standardized",
    stat="probability",
    bins=10,
)

In [ ]:
file_name = f"data/02_postprocessing/{'sample/' if SAMPLE else ''}ratings.csv"
silent_remove(file_name)
with open(file_name, "w") as f:
    rating_df.to_csv(f, index=False, header=True)

## Businesses

### Categories >> Type, Keywords

In [ ]:
with open(f"data/01_cleaned/{'sample/' if SAMPLE else ''}businesses.json") as f:
    businesses = [json.loads(line) for line in f]

In [ ]:
# Count most frequently category occurrences in businesses.json
category_counts = {}
for business in businesses:
    categories_str = business.get("categories", '')
    if categories_str:
        categories = categories_str.split(', ')
        for category in categories:
            category_counts[category] = category_counts.get(category, 0) + 1

# Find the most frequent categories
category_counts = sorted(category_counts.items(), key=lambda x: x[1], reverse=True)
category_counts[:5]

# Create a list of the most frequent categories
top_categories = [category[0] for category in category_counts[:5]]

# print the top 5 most frequent categories
top_categories

In [ ]:
# Keep only the businesses that have at least one of the top 5 categories using a dictionary and create a new attribute "type" for each business that states which of the top 5 categories it belongs to
businesses_dict = {}
for business in businesses:
    categories_str = business.get("categories", '')
    if categories_str:
        categories = categories_str.split(', ')
        for category in categories:
            if category in top_categories:
                business["type"] = category
                businesses_dict[business["business_id"]] = business
                break

In [ ]:
silent_remove(f"data/01_cleaned/{'sample/' if SAMPLE else ''}businesses.json")
with open(f"data/01_cleaned/{'sample' if SAMPLE else ''}/businesses.json", "a") as f:
    for business in businesses_dict.values():
        f.write(json.dumps(business) + "\n")

### Keyword Extraction

In [ ]:
# read business JSON file
with open(f"data/01_cleaned/{'sample/' if SAMPLE else ''}businesses.json") as f:
    businesses = [json.loads(line) for line in f]

In [ ]:
stop_words = set(stopwords.words("english"))
custom_stop_words = [",", "&", "-", "(", ")", ".", "'", "!", "?", ":", ";", "[", "]", "/"]

In [ ]:
# Extract keywords from each business entry for category and name attributes using dictionary
business_keywords = {}
for business in businesses:
    business_keywords[business["business_id"]] = {}
    business_keywords[business["business_id"]]["category"] = []
    business_keywords[business["business_id"]]["name"] = []
    for word in word_tokenize(business["categories"]):
        if word not in stop_words and word not in custom_stop_words:
            business_keywords[business["business_id"]]["category"].append(word)
    for word in word_tokenize(business["name"]):
        if word not in stop_words and word not in custom_stop_words:
            business_keywords[business["business_id"]]["name"].append(word)

In [ ]:
# read review JSON file
with open(f"data/01_cleaned/{'sample/' if SAMPLE else ''}reviews.json") as f:
    reviews = [json.loads(line) for line in f]

In [ ]:
# Extract keywords from each review entry for text attribute using dictionary
review_keywords = {}
for review in reviews:
    review_keywords[review["review_id"]] = []
    for word in word_tokenize(review["text"]):
        if word not in stop_words and word not in custom_stop_words:
            review_keywords[review["review_id"]].append(word)

In [ ]:
# Create a dictionary of business_id to a list of review_ids
business_review_dict = {}
for review in reviews:
    if review["business_id"] not in business_review_dict.keys():
        business_review_dict[review["business_id"]] = [review["review_id"]]
    else:
        business_review_dict[review["business_id"]].append(review["review_id"])

In [ ]:
# combine keywords from all reviews, category, and name for each business
# some businesses do not have any reviews, so we only combine category and name keywords for those businesses using dictionary
business_keywords_combined = {}
for business in businesses:
    business_keywords_combined[business["business_id"]] = []
    if business["business_id"] in business_review_dict.keys():
        for review_id in business_review_dict[business["business_id"]]:
            business_keywords_combined[business["business_id"]] += review_keywords[review_id]
    business_keywords_combined[business["business_id"]] += business_keywords[business["business_id"]]["category"]
    business_keywords_combined[business["business_id"]] += business_keywords[business["business_id"]]["name"]

In [ ]:
# print the keywords for each business with their name
for business in businesses:
    print(business["name"] + ": ")
    print(business_keywords_combined[business["business_id"]])
    print("\n")